In [18]:
import arcpy   
import numpy as np
from osgeo import gdal
import os
import imageio           
import glob  
import shutil

# 1

In [19]:
def ClipFromFolder(raster,inputfolder,outputfolder):
    """从一个文件夹中读取栅格以裁剪另外一个栅格

    Args:
        raster (栅格):待裁剪栅格
        inputfolder (输入文件夹): 包含作为裁剪要素的范围栅格
        outputfolder (输出文件夹): 输出目录
    """
    arcpy.env.workspace = inputfolder
    arcpy.env.overwriteOutput = True
    l = arcpy.ListRasters()
    for i in range(len(l)):
        outraster = outputfolder + "\\" + l[i] 
        description = arcpy.Describe(l[i])
        extent = str(description.extent.XMin) + " " + str(description.extent.YMin) + " " + str(description.extent.XMax) + " " + str(description.extent.YMax)
        arcpy.management.Clip(raster, extent, outraster, l[i], "255", "NONE", "MAINTAIN_EXTENT")
        print( l[i] + ".tif done!")

# 2

In [20]:
def movefile(RefFloder,OriFloder,DesFloder):
    """移动文件，参考RefFloder中的文件，从OriFloder中找到同名文件，移动到DesFloder中
    Args:
        RefFloder (_type_): _description_
        OriFloder (_type_): _description_
        DesFloder (_type_): _description_
    """
    arcpy.env.workspace = RefFloder
    l = arcpy.ListRasters()
    for i in range(len(l)):
        shutil.move(OriFloder + '/' + l[i], DesFloder + "\\" + l[i])

In [21]:
def movefile1(RefFloder,OriFloder,DesFloder,Charter):
    """移动文件，参考RefFloder中的文件，从OriFloder中找到同名文件，移动到DesFloder中，文件具有前缀“Charter_”
    "复制版"
    Args:
        RefFloder (_type_): _description_
        OriFloder (_type_): _description_
        DesFloder (_type_): _description_
    """
    arcpy.env.workspace = RefFloder
    l = arcpy.ListRasters()
    for i in range(len(l)):
        shutil.copy(OriFloder + '/' + Charter + "_" + l[i], DesFloder + "\\" + Charter + "_" + l[i])

In [22]:
def movefile3(OriFloder,DesFloder):
    """移动文件，参考RefFloder中的文件，从OriFloder中找到同名文件，移动到DesFloder中
复制版本
    Args:
        RefFloder (_type_): _description_
        OriFloder (_type_): _description_
        DesFloder (_type_): _description_
    """
    arcpy.env.workspace = OriFloder
    l = arcpy.ListRasters()
    for i in range(len(l)):
        shutil.copy(OriFloder + '/' + l[i], DesFloder + "\\" + l[i])

# 3

In [23]:
def Normalization(inputfolder,outputfolder,upper):
    """
    归一化，upper是上界

    Args:
        inputfolder (_type_): _description_
        outputfolder (_type_): _description_
        upper (_type_): _description_
    """
    oldpath = inputfolder
    newpath = outputfolder
    arcpy.env.workspace = oldpath
    l = arcpy.ListRasters()
    for i in range(len(l)):
        R = arcpy.Raster(oldpath + "\\" + l[i])
        arcpy.CalculateStatistics_management(oldpath + "\\" + l[i])
        k = upper / (R.maximum - R.minimum + 10)
        R1 = k * (R - R.minimum)
        R1.save(newpath + "\\" + l[i])
        print(l[i] + "done!")

# 4

In [24]:
DataFolder = r"D:\proproject\样本制作\ALLData\data"
OtherDataNames = ["C_山体阴影","C_影像","C_Slope"] 
def CompositeBands(DataFolder,OtherDataNames,OutPath):
    """生成多波段影像
        数据类型数量改变时需要更改函数。
    Args:
        DataFolder (_type_): _description_
        OtherDataNames (_type_): _description_
        OutPath (_type_): _description_
    """
    DEM = DataFolder + "\\DEM"
    arcpy.env.workspace = DEM
    l = arcpy.ListRasters()
    names = [j for j in OtherDataNames]
    for i in range(len(l)):
        R = []
        DEM = DataFolder + "\\" + names[0] + "\\" + l[i] 
        Hillshade = DataFolder + "\\" + names[1] + "\\" + l[i]
        Image = DataFolder + "\\" + names[2] + "\\" + l[i]
        Slope = DataFolder + "\\" + names[3] + "\\" + l[i]
        R.append(DEM)
        R.append(Hillshade)
        R.append(Image)
        R.append(Slope)
        if os.path.exists(OutPath + "\\" + l[i]) == False:
            arcpy.CompositeBands_management(R, OutPath + "\\" + l[i])
DEMPath = r"D:\proproject\样本制作\ALLData\data\DEM255"
HillshadePath = r"D:\proproject\样本制作\ALLData\data\山体阴影"
ImagePath = r"D:\proproject\样本制作\ALLData\data\影像"
SlopePath = r"D:\proproject\样本制作\ALLData\data\坡度"
OutputPath = r"D:\proproject\样本制作\ALLData\data\src"
def CompositeBands1(DEMPath,HillshadePath,ImagePath,SlopePath,OutputPath):
    arcpy.env.workspace = DEMPath
    l = arcpy.ListRasters()
    for i in range(len(l)):
        R = []
        DEM = DEMPath + "\\" + l[i] 
        Hillshade = HillshadePath + "\\" + l[i]
        Image = ImagePath + "\\" + l[i]
        Slope = SlopePath + "\\" + l[i]
        R.append(DEM)
        R.append(Hillshade)
        R.append(Image)
        R.append(Slope )
        if os.path.exists(OutputPath + "\\" + l[i]) == False:
            arcpy.CompositeBands_management(R, OutputPath + "\\" + l[i])
CompositeBands1(DEMPath,HillshadePath,ImagePath,SlopePath,OutputPath)

In [25]:
def readtiff2array(oriPath):
    in_ds = gdal.Open(oriPath)
    print("open tif file succeed")

    # 读取原图中的每个波段
    row = in_ds.RasterYSize  # 行
    col = in_ds.RasterXSize  # 列
    band = in_ds.RasterCount  # 波段
    geoTrans = in_ds.GetGeoTransform()
    geoPro = in_ds.GetProjection()
    
    nodatavalue = in_ds.GetRasterBand(1).GetNoDataValue()

    # specific datatype
#     newDataType = gdal.GDT_UInt16;
#     data = np.zeros([row, col, band], newDataType)  # 建立数组保存读取的tiff

    
    # according to the input datatype
    datatype_index = in_ds.GetRasterBand(1).DataType
    datatype = gdal.GetDataTypeName(datatype_index)
#     print(datatype)
    if 'GDT_Byte' in datatype:
        newDataType = 'int8'
    elif 'GDT_UInt16' in datatype:
        newDataType = 'int16'
    else:
        newDataType = 'float32'
    data = np.zeros([row, col, band], newDataType)  # 建立数组保存读取的tiff

    for i in range(band):
        dt = in_ds.GetRasterBand(i + 1)
        # 从每个波段中裁剪需要的矩形框内的数据
        data[:, :, i] = dt.ReadAsArray(0, 0, col, row)             #dataset.ReadAsArray(xoff=0, yoff=0, xsize=512, ysize=512)，前两个参数为该坐标与左上角的像素的相对位置
   
    del in_ds
    
    return data, [band, datatype, geoTrans, geoPro], nodatavalue
    # data shape = HWC

In [26]:
def transfer_16bit_to_8bit(data, isSlope=False, isDEM=False, nodatavalue=91):
    # transform to 8bit

    
    # 以下两部分主要是为了处理无效值，要将无效值区域调至max
    
    ## slope 数据中无效值被设置为了91
    if isSlope is True:
        
        min_16bit = 50000
        max_16bit = 0
    
        # 先对有效值区域进行处理，找到有效值的最大最小值
        ## 找到有效值的所有索引
        indexLS = np.argwhere(data<nodatavalue)
        ## indexLS为tuple，shape=[n,3],n代表有多少行 
        ## 具体在图像中的坐标为 indexLS.shape[0][0], indexLS.shape[0][1]
        rows = indexLS.shape[0]
        i = 0
        for i in range(rows):
            ## 找到其临时值
            tempValue = data[indexLS[i][0],indexLS[i][1],0]
            ## 对临时值进行判断
            if tempValue != 91:
                if tempValue > max_16bit:
                    max_16bit = tempValue
                elif tempValue < min_16bit:
                    min_16bit = tempValue      

        # 进而对无效值（通常为91）进行处理，思路为将其改为最大值+最大值的10%
        # 循环过程与上一步类似
        indexGE = np.argwhere(data>=nodatavalue)    
        rows = indexGE.shape[0]
        i = 0
        for i in range(rows):
            data[indexGE[i][0],indexGE[i][1],0] = max_16bit #+ max_16bit*0.1
        print(max_16bit, min_16bit)
        
    ## dem数据中无效值被设置为了0
    elif isDEM is True:
        nodatavalue = 0
        min_16bit = 50000
        max_16bit = 0
        
        # 先对有效值区域进行处理，找到有效值的最大最小值
        ## 找到有效值的所有索引
        indexGS = np.argwhere(data>nodatavalue)
        ## indexGS为tuple，shape=[n,3],n代表有多少行 
        ## 具体在图像中的坐标为 indexLS.shape[0][0], indexLS.shape[0][1]
        rows = indexGS.shape[0]
        i = 0
        for i in range(rows):
            ## 找到其临时值
            tempValue = data[indexGS[i][0],indexGS[i][1],0]
            ## 对临时值进行判断
            if tempValue != 0:
                if tempValue > max_16bit:
                    max_16bit = tempValue
                elif tempValue < min_16bit:
                    min_16bit = tempValue      

        # 进而对无效值（通常为0）进行处理，思路为将其改为最大值+最大值的10%
        # 循环过程与上一步类似
        indexLE = np.argwhere(data<=nodatavalue)    
        rows = indexLE.shape[0]
        i = 0
        for i in range(rows):
            data[indexLE[i][0],indexLE[i][1],0] = max_16bit #+ max_16bit*0.1
        print(max_16bit, min_16bit)
    else:   
        min_16bit = np.min(data)
        max_16bit = np.max(data)
    
    data_8bit = np.array(np.rint(255 * ((data - min_16bit) / (max_16bit - min_16bit))), dtype=np.uint8)                   #np.rint四舍五入
    return data_8bit

In [27]:
def calculateTransform(ori_transform, offsetX, offsetY):
    # 读取原图仿射变换参数值
    top_left_x = ori_transform[0]  # 左上角x坐标
    w_e_pixel_resolution = ori_transform[1]  # 东西方向像素分辨率
    top_left_y = ori_transform[3]  # 左上角y坐标
    n_s_pixel_resolution = ori_transform[5]  # 南北方向像素分辨率

    # 根据反射变换参数计算新图的原点坐标
    top_left_x = top_left_x + offsetX * w_e_pixel_resolution
    top_left_y = top_left_y + offsetY * n_s_pixel_resolution

    # 将计算后的值组装为一个元组，以方便设置
    dst_transform = (top_left_x, ori_transform[1], ori_transform[2], top_left_y, ori_transform[4], ori_transform[5])

    return dst_transform

In [28]:
def cliptopatch(inputData, ori_geoTrans, ori_geoPro, saveName, offsetCor, patchSize, nodatavalue):
    if os.path.exists(saveName) == False:
        # 读取输入数据信息
        ori_Datatype = inputData.dtype
        # inputData
        if len(inputData.shape) == 3:
            in_bands = inputData.shape[2]
        else:
            in_bands = 1

        # 读取要裁剪的原图
        out_band = np.zeros([patchSize, patchSize, in_bands], ori_Datatype)
        # print(out_band.shape)
        for i in range(in_bands):
            out_band[:, :, i] = inputData[offsetCor[0]:offsetCor[0] + patchSize, offsetCor[1]:offsetCor[1] + patchSize, i]
            out_band[:, :, i] = np.where(out_band[:, :, i]==nodatavalue,-1,out_band[:, :, i])
            
        # 获取原图的原点坐标信息
        ori_transform = ori_geoTrans
        # 计算仿射变化参数
        dst_transform = calculateTransform(ori_transform, offsetCor[1], offsetCor[0])
        # 设置DataType
        if 'int8' in out_band.dtype.name:
            newDataType = gdal.GDT_Byte
        elif 'int16' in out_band.dtype.name:
            newDataType = gdal.GDT_UInt16
        else:
            newDataType = gdal.GDT_Float32
        # 创建gtiff 并 写入
        driver = gdal.GetDriverByName("GTiff")
        
        if not np.any(out_band[:, :, i] == -1) :             #Nodata是-1
            dataset = driver.Create(saveName, patchSize, patchSize, in_bands, newDataType)
            if (dataset != None):
                dataset.SetGeoTransform(dst_transform)  # 写入仿射变换参数
                dataset.SetProjection(ori_geoPro)  # 写入投影
            for i in range(in_bands):
                dataset.GetRasterBand(i + 1).WriteArray(out_band[:, :, i])
            del dataset


In [29]:
def clipgeotiff(inputPath, savePath, patchSize, patchIntersection, startCol, startRow, typeName, index, To8bit=False, isSlope=False,isDEM=False):
    os.makedirs(savePath, exist_ok=True)
    # 遍历文件夹
    from_names = glob.glob(os.path.join(inputPath, "*.tif"))

    # 左上角坐标移动步长
    stride = patchSize - int((patchSize - patchIntersection) / 2)
    print(from_names)
    for i in range(len(from_names)):
        print(from_names[i])
        filePath = os.path.join(inputPath, os.path.basename(from_names[i]))
        # oriPara=[band, datatype, geoTrans, geoPro]
        inputData, oriPara, nodatavalue = readtiff2array(filePath)
        
        # 16bit to 8bit
        if To8bit is True:
            inputData = transfer_16bit_to_8bit(inputData, isSlope,isDEM)

        # 裁剪 起始坐标
        # 原始代码的[offsetY, offsetX] = [offsetRow, offsetCol]
        offsetCor = [startRow, startCol]
        while offsetCor[1] <= inputData.shape[1] - patchSize:
            while offsetCor[0] <= inputData.shape[0] - patchSize:
                filename = typeName + str(index) + '_' + str(offsetCor[1]) + '_' + str(offsetCor[0]) + '.tif'
                saveName = os.path.join(savePath, filename)
                cliptopatch(inputData, oriPara[2], oriPara[3], saveName, offsetCor, patchSize, nodatavalue)
                offsetCor[0] = offsetCor[0] + stride
            offsetCor[1] = offsetCor[1] + stride
            offsetCor[0] = 0
        index = index + 1

# 5 

In [30]:
def Data_augmentation(InputPath,FlipOutPath,MirrorOutPath):
    """对数据进行剪切和镜像增广

    Args:
        InputPath (_type_): 输入文件夹
        FlipOutPath (_type_): 剪切输出
        MirrorOutPath (_type_): 镜像输出
    """
    arcpy.env.workspace = InputPath
    l = arcpy.ListRasters()
    for i in range(len(l)):
        arcpy.management.Mirror(l[i], FlipOutPath + "\\" + l[i])
        arcpy.management.Flip(l[i], MirrorOutPath + "\\" + l[i])
        

# 6

In [21]:
SrcFolder = r"D:\鲁方圆\C_test1\test\src"
DEM_Image_Folder = r"F:\loess不同策略src数据\DEM+影像"
DEM_Image_Slope_Folder = r"F:\loess不同策略src数据\DEM+影像+坡度"
DEM_Image_Hillshade_Folder = r"F:\loess不同策略src数据\DEM+影像+山体阴影"
def Re_CompositeBands(SrcFolder,DEM_Image_Folder,DEM_Image_Slope_Folder,DEM_Image_Hillshade_Folder):
    arcpy.env.workspace = SrcFolder
    l = arcpy.ListRasters()
    for i in range(len(l)):
        R_DEM_Image = []
        R_DEM_Image_Slope = []
        R_DEM_Image_Hillshade = []
        
        src = arcpy.Raster(l[i])
        DEM = src.getRasterBands(1)
        Hillshade = src.getRasterBands(2)
        Imgae_1 = src.getRasterBands(3)
        Imgae_2 = src.getRasterBands(4)
        Imgae_3 = src.getRasterBands(5)
        Slope = src.getRasterBands(6)
        
        R_DEM_Image.append(DEM)
        R_DEM_Image.append(Imgae_1)
        R_DEM_Image.append(Imgae_2)
        R_DEM_Image.append(Imgae_3)
        arcpy.CompositeBands_management(R_DEM_Image, DEM_Image_Folder + "\\" + l[i])
        
        R_DEM_Image_Slope.append(DEM)
        R_DEM_Image_Slope.append(Imgae_1)
        R_DEM_Image_Slope.append(Imgae_2)
        R_DEM_Image_Slope.append(Imgae_3)
        R_DEM_Image_Slope.append(Slope)
        arcpy.CompositeBands_management(R_DEM_Image_Slope, DEM_Image_Slope_Folder + "\\" + l[i])
        
        R_DEM_Image_Hillshade.append(DEM)
        R_DEM_Image_Hillshade.append(Imgae_1)
        R_DEM_Image_Hillshade.append(Imgae_2)
        R_DEM_Image_Hillshade.append(Imgae_3)
        R_DEM_Image_Hillshade.append(Hillshade)
        arcpy.CompositeBands_management(R_DEM_Image_Hillshade, DEM_Image_Hillshade_Folder + "\\" + l[i])

# 7

In [ ]:
def rename(InputPath,Charter):
    """为文件夹下所有文件改名为 “Charter_文件名”
    Args:
        InputPath (_type_): _description_
        Charter (_type_): _description_
    """
    arcpy.env.workspace = InputPath
    l = arcpy.ListRasters()
    for i in range(len(l)):
        shutil.move(InputPath + "\\" + l[i],InputPath + "\\" + Charter + "_" + l[i])
InputPath =  r"D:\proproject\样本制作\样本\样本\C_Samples"
Charter = "O"
#rename(InputPath,Charter)

# 8

In [31]:
def CtreateData(Folder,Strategy,DataFolderNoClip,RefFolder,TrainLabFolder,TestLabFolder,patchSize,patchIntersection):
    """_summary_
        创建数据
        Folder = r"D:\proproject\地貌分类模型训练\不同策略_老样本"  “主文件夹”
        Strategy = r"DEM+影像+山体阴影+坡度"                       “策略”
        DataFolderNoClip = r"D:\proproject\样本制作\ALLData\data\影像和样本\Sample_二分类"    “未裁剪数据”
        RefFolder = r"D:\proproject\地貌分类模型训练\Reference"                             “训练集和测试集的参考，包含test和train，其中各自包含src和lab”
        TrainLabFolder = r"D:\proproject\地貌分类模型训练\Trainning\lab"
        TestLabFolder = r"D:\proproject\地貌分类模型训练\Testing\lab"
        CtreateData(Folder,Strategy,DataFolderNoClip,RefFolder,TrainLabFolder,TestLabFolder,224,200)
    Args:
        Folder (_type_): _description_
        Strategy (_type_): _description_
        DataFolderNoClip (_type_): _description_
        RefFolder (_type_): _description_
        TrainLabFolder (_type_): _description_
        TestLabFolder (_type_): _description_
        patchSize (_type_): _description_
        patchIntersection (_type_): _description_
    """
    arcpy.env.overwriteOutput = True
    StrategyFolder = Folder + "\\" +Strategy
    if not os.path.exists(StrategyFolder):
        os.mkdir(StrategyFolder)
    SrcFolder = StrategyFolder + "\\Data"
    if not os.path.exists(SrcFolder):
        os.mkdir(SrcFolder)
    OriginFolder = SrcFolder + "\\origin"
    if not os.path.exists(OriginFolder):
        os.mkdir(OriginFolder)
    FlipFolder = SrcFolder + "\\flip"
    if not os.path.exists(FlipFolder):
        os.mkdir(FlipFolder)
    MirrorFolder = SrcFolder + "\\mirror"
    if not os.path.exists(MirrorFolder):
        os.mkdir(MirrorFolder)
        
    OriginFolderClip = SrcFolder + "\\C_origin"
    if not os.path.exists(OriginFolderClip):
        os.mkdir(OriginFolderClip)
    FlipFolderClip = SrcFolder + "\\C_flip"
    if not os.path.exists(FlipFolderClip):
        os.mkdir(FlipFolderClip)    
    MirrorFolderClip = SrcFolder+ "\\C_mirror"
    if not os.path.exists(MirrorFolderClip):
        os.mkdir(MirrorFolderClip)    
      
    TrainFolder = StrategyFolder + "\\Train"
    Train_src = TrainFolder + "\\src"
    Train_lab = TrainFolder + "\\lab"
    print(Train_lab)
    if not os.path.exists(TrainFolder):
        os.mkdir(TrainFolder)  
        os.mkdir(Train_lab)
        os.mkdir(Train_src)
        
    TestFolder = StrategyFolder + "\\Test"
    Test_src = TestFolder + "\\src"
    Test_lab = TestFolder + "\\lab"
    if not os.path.exists(TestFolder):
        os.mkdir(TestFolder)  
        os.mkdir(Test_lab)
        os.mkdir(Test_src)
        
    movefile3(DataFolderNoClip,OriginFolder)
    Data_augmentation(OriginFolder,FlipFolder,MirrorFolder)

    startCol = 0
    startRow = 0
    typeName = 'W'
    index = 0
    clipgeotiff(OriginFolder, OriginFolderClip, patchSize, patchIntersection, startCol, startRow, typeName, index, To8bit=False, isSlope=False,isDEM=True)
    clipgeotiff(FlipFolder, FlipFolderClip, patchSize, patchIntersection, startCol, startRow, typeName, index, To8bit=False, isSlope=False,isDEM=True)
    clipgeotiff(MirrorFolder, MirrorFolderClip, patchSize, patchIntersection, startCol, startRow, typeName, index, To8bit=False, isSlope=False,isDEM=True)
    
    rename(OriginFolderClip,"O")
    rename(FlipFolderClip,"F")
    rename(MirrorFolderClip,"M")
    
    
    Train_RefSrc = RefFolder + "\\Train\\src"  
    
    movefile1(Train_RefSrc,OriginFolderClip,Train_src,"O")
    movefile1(Train_RefSrc,FlipFolderClip,Train_src,"F")
    movefile1(Train_RefSrc,MirrorFolderClip,Train_src,"M")
    
    Test_RefSrc = RefFolder + "\\Test\\src"  
    
    movefile1(Test_RefSrc,OriginFolderClip,Test_src,"O")
    movefile1(Test_RefSrc,FlipFolderClip,Test_src,"F")
    movefile1(Test_RefSrc,MirrorFolderClip,Test_src,"M")
    
    # TrainLabFolder = RefFolder + "\\Train\\lab"
    # TestLabFolder = RefFolder + "\\Test\\lab"
    movefile3(TrainLabFolder,Train_lab)
    movefile3(TestLabFolder,Test_lab)